<a href="https://colab.research.google.com/github/Shiv2183/StaffOptimization/blob/main/Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install pyomo
!sudo apt install libglpk-dev python3.6-dev libgmp3-dev
!pip install glpk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgmp3-dev is already the newest version (2:6.1.2+dfsg-2).
libglpk-dev is already the newest version (4.65-1).
python3.6-dev is already the newest version (3.6.9-1~18.04ubuntu1.3).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [28]:
!git clone https://github.com/Shiv2183/StaffOptimization.git

fatal: destination path 'StaffOptimization' already exists and is not an empty directory.


In [29]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [30]:
from __future__ import division
from pyomo.opt import SolverFactory

In [31]:
# Reading the data from Excel workbook

In [32]:
demand = pd.read_excel('/content/StaffOptimization/Staffing+Data.xlsx', sheet_name =0)
StaffAvailability = pd.read_excel('/content/StaffOptimization/Staffing+Data.xlsx', sheet_name =1)
Cost = pd.read_excel('/content/StaffOptimization/Staffing+Data.xlsx', sheet_name =2)
ServiceRate = pd.read_excel('/content/StaffOptimization/Staffing+Data.xlsx', sheet_name =3)

In [33]:
demand.head(3)

,State,Month,Demand
0,A,Jan,5240
1,A,Feb,4878
2,A,Mar,5942


In [34]:
StaffAvailability.head(3)

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75


In [35]:
Cost.head(3)

,State,Month,AnnualSalary,MonthlySalary,UnitOutSourceCost
0,A,Jan,60000,5000.0,180
1,A,Feb,60000,5000.0,180
2,A,Mar,60000,5000.0,180


In [ ]:
ServiceRate.head(3)

,MgAppServedPerMonth
0,40


# Data pre-processing

In [ ]:
# Create the required Python data structures for indexes and parameters

In [52]:
#Merge demand, StaffAvailability and Cost
comb_df = pd.merge(demand, StaffAvailability, on =['State', 'Month'], how='left')
comb_df = pd.merge(comb_df, Cost, on =['State', 'Month'], how='left')
#Create FTE cost during worst case (LB) and best case (UB) and Average case(StaffAvPer)
comb_df['LB_UnitFTECost'] = (comb_df['MonthlySalary']/ServiceRate.iloc[0, 0])*(1/comb_df['LB'])
comb_df['UB_UnitFTECost'] = (comb_df['MonthlySalary']/ServiceRate.iloc[0, 0])*(1/comb_df['UB'])
comb_df['Avg_UnitFTECost'] = (comb_df['MonthlySalary']/ServiceRate.iloc[0, 0])*(1/comb_df['StaffAvPer'])
#Calculate Maximum demand which can be outsourced
comb_df['Max_Demand_outsourced'] = np.where(comb_df['State']=='A',comb_df['Demand'] * 0.4, np.where(comb_df['State']=='B',comb_df['Demand'] * 0.3, comb_df['Demand']))
comb_df

,State,Month,Demand,LB,UB,StaffAvPer,AnnualSalary,MonthlySalary,UnitOutSourceCost,LB_UnitFTECost,UB_UnitFTECost,Avg_UnitFTECost,Max_Demand_outsourced
0,A,Jan,5240,0.70,0.90,0.81,60000,5000.000000,180,178.571429,138.888889,154.320988,2096.0
1,A,Feb,4878,0.65,0.85,0.76,60000,5000.000000,180,192.307692,147.058824,164.473684,1951.2
2,A,Mar,5942,0.70,0.80,0.75,60000,5000.000000,180,178.571429,156.250000,166.666667,2376.8
3,A,Apr,2297,0.75,0.85,0.80,60000,5000.000000,180,166.666667,147.058824,156.250000,918.8
4,A,May,1992,0.70,0.85,0.78,60000,5000.000000,180,178.571429,147.058824,160.256410,796.8
5,A,Jun,2275,0.65,0.80,0.73,60000,5000.000000,180,192.307692,156.250000,171.232877,910.0
6,A,Jul,5334,0.60,0.75,0.68,60000,5000.000000,180,208.333333,166.666667,183.823529,2133.6
7,A,Aug,3371,0.65,0.85,0.76,60000,5000.000000,180,192.307692,147.058824,164.473684,1348.4
8,A,Sep,3759,0.70,0.90,0.81,60000,5000.000000,180,178.571429,138.888889,154.320988,1503.6
9,A,Oct,3529,0.65,0.80,0.73,60000,5000.000000,180,192.307692,156.250000,171.232877,1411.6


# Question 1
The company wants to know the optimised staffing recommendations for the business case described. Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).

**Note:** You can write the mathematical formulation directly in your report.

# Question 2
Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step.

**Expected output:**
Create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier.

In [53]:
# Creating a model instance
M = ConcreteModel()

In [56]:
# Define Pyomo sets and Parameters
demand_fte = []
demand_outsourced = []
M.T = Set(doc='month', initialize=comb_df.index.tolist(), ordered=True)

In [57]:
# Decision variables
M.demand_fte = Var(M.T, domain=NonNegativeReals, initialize=0)
M.demand_outsourced = Var(M.T, domain=NonNegativeReals, initialize=0)

In [62]:
!apt-get install -y -qq glpk-utils
# Constraints & Objective function
for t in range(len(comb_df)):
  #Constraints
  M.totaldemand=Constraint(expr=(M.demand_fte[t] + M.demand_outsourced[t])==comb_df['Demand'][t])
  M.max_outsource_demand = Constraint(expr=(M.demand_outsourced[t])<=comb_df['Max_Demand_outsourced'][t])

  #Objective Function
  expression = M.demand_fte[t] * comb_df['Avg_UnitFTECost'][t] + M.demand_outsourced[t]* comb_df['UnitOutSourceCost'][t]
  M.obj = Objective(expr=expression ,sense= minimize)

  #Optimizer
  optimizer=SolverFactory('glpk', executable='/usr/bin/glpsol')
  optimizer.solve(M)

  demand_fte.append(value(M.demand_fte[t]))
  demand_outsourced.append(value(M.demand_outsourced[t]))

    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.AbstractSimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.constraint.SimpleConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.AbstractSimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=

In [ ]:
# Invoking the solver

In [ ]:
# Print the value of the objective function
(demand_fte*comb_df['Avg_UnitFTECost'] + demand_outsourced * comb_df['UnitOutSourceCost']).sum()

**Checkpoint 1:** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [ ]:
# Creating dataframe for the results


In [ ]:
# Writing the results in to an Excel sheet

# Question 3

**Worst-case and best-case analysis based on the staffs' availability.**
Assuming that the distribution is the same across all the states,

**3.1 Worst case analysis**

*   3.1.1 What is the optimal number of staff members for the worst case? 
*   3.1.2 What is the percentage of outsourcing for the worst case?
*   3.1.3 What is the average cost per application for the worst case?

**3.2 Best case analysis**


*   3.2.1 What is the optimal number of staff members List item 
for the best case?
*   3.2.2 What is the percentage of outsourcing for the best case? 
*   3.2.3 What is the average cost per application for the best case?

**Expected output:**
For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application.

#### 3.1 Worst Case Analysis
**3.1.1 Optimal number of staff members**

In [ ]:
# Creating a model instance

In [ ]:
# Define Pyomo sets and Parameters

In [ ]:
# Decision variables

In [ ]:
# Constraints

In [ ]:
# Objective function

In [ ]:
# Invoking the solver

In [ ]:
# Print the value of the objective function

**Checkpoint 2: The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.**

In [ ]:
# Creating dataframe for the results

**3.1.2 Percentage of outsourced applications**

In [ ]:
# write your code here

**3.1.3 Average cost per application**

In [ ]:
# write your code here

#### 3.2 Best Case Analysis

**3.2.1 Optimal number of staff members**

In [ ]:
# Creating a model instance

In [ ]:
# Define Pyomo sets and Parameters

In [ ]:
# Decision variables

In [ ]:
# Constraints

In [ ]:
# Objective function

In [ ]:
# Invoking the solver

In [ ]:
# Print the value of the objective function

**Checkpoint 3: The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.**

In [ ]:

# Creating dataframe for the results

**3.2.2 Percentage of outsourced applications**

In [ ]:
# write your code here

**3.2.3 Average cost per application**

In [ ]:
# write your code here

# Question 4
**Creating Visualisations**
Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%).  Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis. Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.